In [27]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import lxml

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd

from datetime import datetime

#Отображение колонок и строк в VScode
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [25]:
profs_all = pd.read_csv('profs.txt', header=None, names=['profs_names'])
profs_test = pd.read_csv('profs_test.txt', header=None, names=['profs_names'])

In [28]:
tinkoff_job = pd.DataFrame(columns=['link', 'name', 'location', 'company', 'date', 'description', 'date_of_download'])
df_test = pd.DataFrame(columns=['link', 'name', 'location', 'company', 'date', 'description', 'date_of_download'])

In [93]:
from urllib.request import urlopen

url = urlopen('https://www.tinkoff.ru/career/it/')
soup = BeautifulSoup(url.read(), 'lxml')

headers = {}
rows = soup.find_all("tr")
thead = soup.find_all("th")

for i in range(len(thead)):
     headers[i] = thead[i].text.strip().lower()

data = []

for row in rows:
     cells = row.find_all("td")

item = {}

for index in headers:
     item[headers[index]] = cells[index].text
     data.append(item)

print(data)


[]


In [50]:
class TinkoffJobParser:

    url = 'https://www.tinkoff.ru/career/it/'
    
    def __init__(self):
        self.browser = webdriver.Chrome()
        self.browser.get(self.url)
        self.browser.maximize_window()
        self.browser.delete_all_cookies()
        time.sleep(2)

    def scroll_page(self, page_height=0):
        to_button = self.browser.find_element(By.CLASS_NAME, 'b5VfE3')
        self.browser.execute_script("return arguments[0].scrollIntoView(true);", to_button)
        
    def stop(self):
        '''
        Выйти из Webdriver selenium
        '''
        self.browser.quit()
    
    def find_vacancies(self):
        '''
        Функция для ввода названия вакансии (vac_type) в поисковую строку и клика клавиши Enter
        '''
        try:
            # находим количество строк в таблице
            rows = len(self.browser.find_elements(by=By.XPATH, value = '/html/body/div[1]/div/div[3]/div/div[2]/div[1]/div[1]/table/tbody/tr[1]'))
            # подсчет количества столбцов
            cols = len(self.browser.find_elements(by=By.XPATH, value = '/html/body/div[1]/div/div[3]/div/div[2]/div[1]/div[1]/table/tbody/tr[1]/td[1]'))
            

        except:
            #input_button.clear()
            pass
            
        df_test.drop_duplicates(subset=['link'], keep='last')

    def find_description(self):
        for descr in range(len(df_test)):
            try:
                link = df_test.loc[descr, 'link']
                self.browser.get(link)
                self.browser.delete_all_cookies()
                time.sleep(3)

                df_test.loc[descr, 'description'] = self.browser.find_element(By.CLASS_NAME, 'section').text
            except:
                pass

    def save_df(self):
        df_test.to_csv(f"all_{datetime.now()}.txt", index=False, sep=';')

In [51]:
parser = TinkoffJobParser()
parser.find_vacancies()
parser.stop()

In [39]:
parser = TinkoffJobParser(profs_all)
parser.find_description()
parser.stop()

TypeError: TinkoffJobParser.__init__() takes 1 positional argument but 2 were given

In [49]:
df_test

,link,name,location,company,date,description,date_of_download


In [109]:
df_test.to_csv(f"all_vk.txt", index=False, sep='(/)')